In [37]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import neuropythy as ny
import matplotlib.pyplot as plt

# personal import 
from math_utils import weighted_regression, weighted_nan_percentile, weighted_nan_median


In [2]:
# Settings
num_ecc_pcm_bins = 15
max_ecc = 8
rois = ['V1', 'V2', 'V3']

In [3]:
# Get subject numbers
allsub = ny.data['hcp_lines'].subject_list

In [4]:
path = '/home/jovyan/projects/pRF-project_NH2025/data/tsv/group'
grouptsv = pd.read_csv('{}/group_prf_parameters_5mm.tsv'.format(path), sep='\t')
grouptsv

,prf_dist,geo_dist,prf_ecc,prf_r2,hemi,roi,subject,pRF_CM
0,0.216579,3.131530,0.456203,0.400212,rh,V1,100610,14.459055
1,0.170155,3.137668,0.456203,0.400212,rh,V1,100610,18.440021
2,0.263003,3.180024,0.456203,0.400212,rh,V1,100610,12.091209
3,0.263003,3.264105,0.456203,0.400212,rh,V1,100610,12.410903
4,0.216579,3.391770,0.456203,0.400212,rh,V1,100610,15.660649
...,...,...,...,...,...,...,...,...
1960089,1.283459,3.382235,5.451297,0.549699,lh,V3,901442,2.635249
1960090,2.178203,3.287114,5.733670,0.485306,lh,V3,901442,1.509095
1960091,1.106994,3.325491,5.921214,0.549911,lh,V3,901442,3.004075
1960092,1.277594,3.562248,5.921214,0.549911,lh,V3,901442,2.788247


In [11]:
ecc_bins = np.concatenate(([0], np.logspace(np.log10(0.1), np.log10(max_ecc), num=num_ecc_pcm_bins)))

start_time = time.time()
group_bin_df = pd.DataFrame()
for n_sub, subject_id in enumerate(allsub):
    for num_roi, roi in enumerate(rois):
        df_subject_roi = grouptsv.loc[(grouptsv['roi'] == roi) & (grouptsv['subject'] == subject_id)]
        df_bins = df_subject_roi.groupby(pd.cut(df_subject_roi['prf_ecc'], bins=ecc_bins))
        
        df_subject_roi_bin = pd.DataFrame()
        df_subject_roi_bin['roi'] = [roi]*num_ecc_pcm_bins
        df_subject_roi_bin['subject_id'] = [subject_id]*num_ecc_pcm_bins
        
        df_subject_roi_bin['num_bins'] = np.arange(num_ecc_pcm_bins)
        
        df_subject_roi_bin['prf_ecc_bins'] = df_bins.apply(lambda x: weighted_nan_median(x['prf_ecc'].values, x['prf_r2'].values)).values
        df_subject_roi_bin['prf_pcm_bins_median'] = df_bins.apply(lambda x: weighted_nan_median(x['pRF_CM'].values, x['prf_r2'].values)).values
        df_subject_roi_bin['prf_r2_bins_median'] = np.array(df_bins['prf_r2'].median())

    
        df_subject_roi_bin['prf_pcm_bins_ci_upper_bound'] = df_bins.apply(lambda x: weighted_nan_percentile(x['pRF_CM'].values, x['prf_r2'].values, 75)).values
        df_subject_roi_bin['prf_pcm_bins_ci_lower_bound'] = df_bins.apply(lambda x: weighted_nan_percentile(x['pRF_CM'].values, x['prf_r2'].values, 25)).values
        
        if num_roi == 0: df_ecc_pcm_bins = df_subject_roi_bin
        else: df_ecc_pcm_bins = pd.concat([df_ecc_pcm_bins, df_subject_roi_bin]) 
            
    group_bin_df = pd.concat([group_bin_df, df_ecc_pcm_bins])
end_time = time.time()
print("Execution time: {:.2f} seconds".format(end_time - start_time))

Execution time: 178.49 seconds


In [12]:
group_bin_df

,roi,subject_id,num_bins,prf_ecc_bins,prf_pcm_bins_median,prf_r2_bins_median,prf_pcm_bins_ci_upper_bound,prf_pcm_bins_ci_lower_bound
0,V1,100610,0,0.060130,23.146870,0.069411,24.253378,18.827737
1,V1,100610,1,0.111363,40.533411,0.159071,43.688370,13.849380
2,V1,100610,2,0.163424,28.632962,0.208144,34.740110,21.943321
3,V1,100610,3,0.206557,27.240073,0.309376,38.469654,21.411126
4,V1,100610,4,0.302808,20.108544,0.284413,22.141127,17.140909
...,...,...,...,...,...,...,...,...
10,V3,901442,10,1.908191,4.186478,0.412553,6.072827,3.250407
11,V3,901442,11,2.656185,3.464509,0.387613,4.336131,2.685188
12,V3,901442,12,3.499824,2.795083,0.440295,4.127043,2.123320
13,V3,901442,13,5.175815,1.841641,0.452772,2.313703,1.494051


In [21]:
group_subid_roi_df

In [34]:
final_group_df = pd.DataFrame()

for n_roi, roi in enumerate(rois):
    final_group_df_roi = pd.DataFrame()
    df_roi = group_bin_df.loc[(group_bin_df['roi'] == roi)]
    group_subid_roi_df = df_roi.groupby(['num_bins'])
    
    final_group_df_roi['roi'] = [roi]*num_ecc_pcm_bins
    final_group_df_roi['prf_ecc_bins'] = group_subid_roi_df.apply(lambda x: weighted_nan_median(x['prf_ecc_bins'].values, x['prf_r2_bins_median'].values)).values
    final_group_df_roi['prf_pcm_bins_median'] = group_subid_roi_df.apply(lambda x: weighted_nan_median(x['prf_pcm_bins_median'].values, x['prf_r2_bins_median'].values)).values
    final_group_df_roi['prf_r2_bins_median'] = np.array(group_subid_roi_df['prf_r2_bins_median'].median())
    
    final_group_df = pd.concat([final_group_df, final_group_df_roi])

final_group_df = final_group_df.reset_index().rename(columns={'index': 'num_bin'})
   

In [35]:
final_group_df

,num_bin,roi,prf_ecc_bins,prf_pcm_bins_median,prf_r2_bins_median
0,0,V1,0.088160,18.318069,0.087329
1,1,V1,0.127362,19.911860,0.115069
2,2,V1,0.164440,17.114455,0.119276
3,3,V1,0.224154,18.457760,0.150333
4,4,V1,0.307375,15.682026,0.185672
5,5,V1,0.419535,13.426744,0.238485
6,6,V1,0.563041,11.681930,0.286856
7,7,V1,0.743892,9.299545,0.342006
8,8,V1,1.117013,7.411009,0.398038
9,9,V1,1.495909,6.000981,0.458200


In [38]:
# Export DF 
tsv_dir = '/home/jovyan/projects/pRF-project_NH2025/data/tsv/group'
os.makedirs(tsv_dir, exist_ok=True)

tsv_fn = '{}/final_group_df.tsv'.format(tsv_dir)
tsv_fn2 = '{}/group_bin_df.tsv'.format(tsv_dir)
group_bin_df.to_csv(tsv_fn2, sep="\t", na_rep='NaN', index=False)
final_group_df.to_csv(tsv_fn, sep="\t", na_rep='NaN', index=False)